In [1]:
import open3d as o3d
import cv2
import numpy as np
import time
from apriltag import apriltag


In [2]:
color_image = cv2.imread('/home/yunke/3dv_proj/bias correction/scan0512/1/1/rgb/color0004.png',cv2.IMREAD_GRAYSCALE)
detector = apriltag("tag36h11")
result = detector.detect(color_image)
cv2.imwrite("/home/yunke/color.png",color_image)
print(result)

({'hamming': 0, 'margin': 126.58293914794922, 'id': 1, 'center': array([702.05119353, 882.71382462]), 'lb-rb-rt-lt': array([[647.71026611, 930.00708008],
       [740.80383301, 926.8850708 ],
       [752.65447998, 838.67346191],
       [665.65740967, 841.23126221]])}, {'hamming': 0, 'margin': 122.68973541259766, 'id': 4, 'center': array([1045.16847508,  875.04402227]), 'lb-rb-rt-lt': array([[1004.61523438,  920.38061523],
       [1094.31140137,  919.2010498 ],
       [1083.06567383,  832.67675781],
       [ 999.11645508,  833.66430664]])}, {'hamming': 0, 'margin': 131.10140991210938, 'id': 6, 'center': array([611.31789673, 780.05984421]), 'lb-rb-rt-lt': array([[557.83319092, 820.10876465],
       [644.7210083 , 817.54547119],
       [661.29833984, 742.6348877 ],
       [579.71002197, 744.58886719]])}, {'hamming': 0, 'margin': 124.2203140258789, 'id': 8, 'center': array([826.25530645, 774.97928441]), 'lb-rb-rt-lt': array([[781.33001709, 813.89941406],
       [865.09289551, 812.08172607],

In [ ]:
depth_image = cv2.imread('/home/yunke/3dv_proj/bias correction/scan0512/1/1/depth/depth0001.png')
print(depth_image.shape)
print(np.max(depth_image))

In [ ]:
source_color = o3d.io.read_image("/home/yunke/3dv_proj/bias correction/scan0512/color.png")
source_depth = o3d.io.read_image("/home/yunke/3dv_proj/bias correction/scan0512/1/1/depth/depth0001.png")
source_rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        source_color, source_depth)
intrinsics = [640, 576, 503.927643, 503.9782729, 332.06427, 325.880615]


In [ ]:
def get_central_plane(pcd, plane, fx, fy, cx, cy, inlier):
    points = np.asarray(pcd.select_down_sample(inlier).points)
    central_points = []
#     for i in range(points.shape[0]):
#         x = points[i][0]
#         y = points[i][1]
#         z = points[i][2]
#         #project point on image
#         u = int(cx + x*fx/z)
#         v = int(cy + y*fy/z)
#         if abs(u-cx)<50 and abs(v-cy)<30:
#             central_points.append(np.asarray([x,y,z]))
    cam_mat = np.asarray([[fx,0,cx],
                         [0,fy,cy],
                         [0,0,1]])
    z = points[:,2]
    #print(z.shape)
    z_u_v_1 = np.transpose(np.dot(cam_mat,np.transpose(points)))
    #print(z_u_v_1)
    u_v_1 = z_u_v_1/z[:,None]
    u_v_1 = u_v_1.astype(int)
    u = u_v_1[:,0]
    v = u_v_1[:,1]
    central_x = abs(u-cx)<50 
    central_y = abs(v-cy)<30
    central = np.concatenate([central_x.reshape((-1,1)),central_y.reshape((-1,1))],axis = 1)
    #print(central.shape)
    central = central.all(axis = 1)
    print(central.shape)
    
    seq = np.arange(u.shape[0])
    seq = seq[central]
    #plane line intersect
    central_points = points[seq,:]
    n = np.asarray(plane)[0:3]
    
    ds = -np.dot(central_points, n)
    d = np.mean(ds)
    new_plane = np.concatenate([n,[d]])
    return new_plane

In [ ]:
def pcd_to_err(pcd, plane, fx, fy, cx, cy, inlier):
    points = np.asarray(pcd.select_down_sample(inlier).points)
    data = []
#     for i in range(points.shape[0]):
#         x = points[i][0]
#         y = points[i][1]
#         z = points[i][2]
#         #project point on image
#         u = int(cx + x*fx/z)
#         v = int(cy + y*fy/z)
#         #plane line intersect
#         n = np.asarray(plane)[0:3]
#         d = np.asarray(plane)[3]
#         l = np.asarray([float(u-cx)/fx,float(v-cy)/fy,1])
#         z_t = -d/np.dot(l,n)
#         if z_t<1.5:
#             data.append((u,v,z,z_t))
    cam_mat = np.asarray([[fx,0,cx],
                         [0,fy,cy],
                         [0,0,1]])
    z = points[:,2]
    #print(z.shape)
    z_u_v_1 = np.transpose(np.dot(cam_mat,np.transpose(points)))
    #print(z_u_v_1)
    u_v_1 = z_u_v_1/z[:,None]
    u_v_1 = u_v_1.astype(int)
    #print(u_v_1.shape)
    #print(u_v_1)
    l = np.transpose(np.dot(np.linalg.inv(cam_mat),np.transpose(u_v_1)))
    n = np.asarray(plane)[0:3]
    d = np.asarray(plane)[3]
    z_t = -d/np.dot(l,n)
    
    seq = np.arange(z.shape[0])
    seq = seq[z<1.5]
    data = list(zip(u_v_1[seq,0].tolist(),u_v_1[seq,1].tolist(),z[seq].tolist(),z_t[seq].tolist()))
    
    #print(data)
    return data



In [ ]:
def source_img_to_data(source_color, source_depth, intrinsic, training_data):
    source_rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        source_color, source_depth)
    intrinsics = o3d.camera.PinholeCameraIntrinsic()
    intrinsics.set_intrinsics(intrinsic[0], intrinsic[1], intrinsic[2], intrinsic[3], intrinsic[4], intrinsic[5])
    # intrinsics = np.asarray([[503.927643,0,332.06427],
    #                         [0,503.9782729,325.880615],
    #                         [0,0,1]])

    pcd = o3d.geometry.PointCloud.create_from_depth_image(
            source_depth, intrinsics)
    #o3d.visualization.draw_geometries([pcd])
    #segment plane
    plane, inlier = pcd.segment_plane(0.01, 20, 1000)
    #print(plane)
    #get plane on central points
    new_plane = get_central_plane(pcd, plane, intrinsic[2], intrinsic[3], intrinsic[4], intrinsic[5], inlier)
    print(plane)
    print(new_plane)
    #print(inlier)
    #o3d.visualization.draw_geometries([pcd.select_down_sample(inlier)])
    data = pcd_to_err(pcd, new_plane, intrinsic[2], intrinsic[3], intrinsic[4], intrinsic[5], inlier)
    # Blur the image
    source_depth_blur = cv2.bilateralFilter(np.asarray(source_depth).astype(np.uint8),9,100,100)
    
    sobelx = cv2.Sobel(np.asarray(source_depth_blur),cv2.CV_64F,1,0,ksize=5)
    sobely = cv2.Sobel(np.asarray(source_depth_blur),cv2.CV_64F,0,1,ksize=5)
    #print(sobelx.shape)3
    #print(np.asarray(source_depth).shape)
    print(np.max(sobely))
    for i in range(len(data)):
        u = data[i][0]
        v = data[i][1]
        sobelx_i = sobelx[v][u]
        sobely_i = sobely[v][u]
        
        if abs(sobelx_i)<500 and abs(sobely_i)<500:
            temp = list(data[i])
            temp.append(sobelx_i)
            temp.append(sobely_i)
            training_data.append(tuple(temp))

    

In [ ]:
import os

path = "/home/yunke/3dv_proj/bias correction/scan0512"
filelist = []
for root, dirs, files in os.walk(path):
    for file in files:
        #append the file name to the list
        if file[0:6]=="depth0":
            filelist.append(os.path.join(root,file))
print(filelist)

In [ ]:
source_color = o3d.io.read_image("/home/yunke/3dv_proj/bias correction/scan0512/color.png")
intrinsics = [640, 576, 503.927643, 503.9782729, 332.06427, 325.880615]
training_data = []
for img_path in filelist:
    source_depth = o3d.io.read_image(img_path)    
    source_img_to_data(source_color, source_depth, intrinsics, training_data)
    
print(len(training_data))

In [ ]:
print(training_data[0:10])

In [ ]:
with open('/home/yunke/3dv_proj/bias correction/scan0512/training_data.txt', 'w') as fp:
    fp.write('\n'.join('%s %s %s %s %s %s' % x for x in training_data))